# Init

In [0]:
import pyspark.sql.functions as F
from pyspark.sql.types import StringType, DateType
from pyspark.sql.functions import trim, col

# Reading From Bronze Table

In [0]:
df = spark.table("workspace.bronze.crm_prd_info")

# Data Transformations

## Deleting leading spaces - trimming the values

In [0]:
for field in df.schema.fields:
    if isinstance(field.dataType, StringType):
        df = df.withColumn(field.name, trim(col(field.name)))

## product line normalization

In [0]:
df = (
    df
    .withColumn(
        "prd_line",
        F.when(F.upper(col("prd_line")) == "R", "Road")
        .when(F.upper(col("prd_line")) == "S", "Other Sales")
        .when(F.upper(col("prd_line")) == "M", "Mountain")
        .when(F.upper(col("prd_line")) == "T", "Touring")
        .otherwise("n/a")
    )
)

## cost normalization - taking absolute number for each row (nulls will be converted to 0)

In [0]:
df = df.withColumn("prd_cost", F.coalesce(col("prd_cost"), F.lit(0)))

## Product key parsing

In [0]:

df = df.withColumn("cat_id", F.regexp_replace(F.substring(col("prd_key"), 1, 5), "-", "_"))
df = df.withColumn("prd_key", F.substring(col("prd_key"), 7, F.length(col("prd_key"))))

## Renaming Columns

In [0]:
RENAME_MAP = {
    "prd_id": "product_id",
    "prd_key": "product_key",
    "prd_nm": "product_name",
    "prd_cost": "product_cost",
    "prd_line": "product_line",
    "prd_start_dt": "start_date",
    "prd_end_dt": "end_date",
    "cat_id": "category_id"
}

In [0]:
for old_name, new_name in RENAME_MAP.items():
    df = df.withColumnRenamed(old_name, new_name)

# Sanity check before writing to silver delta table

In [0]:
df.display()

# Writing to Silver table

In [0]:
df.write.mode("overwrite").format("delta").saveAsTable("workspace.silver.crm_products")

# Sanity check for silver table

In [0]:
%sql
SELECT * FROM workspace.silver.crm_products